In [ ]:
# Install necessary libraries
!pip install torch transformers datasets accelerate peft bitsandbytes

hiRequirement already satisfied: torch in /usr/local/lib/python3.11/dist-packages (2.6.0+cu124)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 63.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 45.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 33.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 87.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 29.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer
from datasets import load_dataset
from peft import get_peft_model, LoraConfig, TaskType

In [ ]:
# ✅ Load tokenizer
model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
# ✅ Load dataset (Alpaca)
dataset = load_dataset("tatsu-lab/alpaca", split="train")

# ✅ Function to format dataset correctly
def format_data(sample):
    # Prepare prompt
    prompt = f"Instruction: {sample['instruction']}\nResponse:"

    # Tokenize inputs
    inputs = tokenizer(prompt, truncation=True, padding="max_length", max_length=512)

    # Tokenize labels (expected response)
    labels = tokenizer(sample["output"], truncation=True, padding="max_length", max_length=512)["input_ids"]

    # Mask padding tokens in labels to ignore them in loss computation
    labels = [-100 if token == tokenizer.pad_token_id else token for token in labels]

    return {
        "input_ids": inputs["input_ids"],
        "attention_mask": inputs["attention_mask"],
        "labels": labels
    }

# ✅ Apply formatting
dataset = dataset.map(format_data, remove_columns=["instruction", "output", "input", "text"])

Map:   0%|          | 0/52002 [00:00<?, ? examples/s]

In [ ]:
# ✅ Split into 90% train & 10% eval
split_dataset = dataset.train_test_split(test_size=0.1)
train_dataset = split_dataset["train"]
eval_dataset = split_dataset["test"]

In [ ]:
# ✅ Load TinyLlama Model (FP16 for memory efficiency)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16, device_map="auto")

In [ ]:
# ✅ LoRA Configuration
lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    r=1,
    lora_alpha=32,
    lora_dropout=0.05,
    target_modules=["q_proj", "v_proj"]
)

# ✅ Apply LoRA to the model
model = get_peft_model(model, lora_config)

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=1,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_dir="./logs",
    logging_strategy="epoch",
    report_to="none",  # ✅ Disables W&B completely
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
# initialize trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
)

# start fine tuning
trainer.train()

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Epoch,Training Loss,Validation Loss


Epoch,Training Loss,Validation Loss


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel
import torch

# Set paths
base_model_path = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"  # Base model from HF
fine_tuned_model_path = "./results/checkpoint-5851"  # Your LoRA fine-tuned model

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model_path)

# Load base model
model = AutoModelForCausalLM.from_pretrained(base_model_path, torch_dtype=torch.float16, device_map="auto")

# Load fine-tuned LoRA adapter
model = PeftModel.from_pretrained(model, fine_tuned_model_path)


/usr/local/lib/python3.11/dist-packages/peft/peft_model.py:599: UserWarning: Found missing adapter keys while loading the checkpoint: ['base_model.model.model.layers.0.self_attn.q_proj.lora_A.default.weight', 'base_model.model.model.layers.0.self_attn.q_proj.lora_B.default.weight', 'base_model.model.model.layers.0.self_attn.v_proj.lora_A.default.weight', 'base_model.model.model.layers.0.self_attn.v_proj.lora_B.default.weight', 'base_model.model.model.layers.1.self_attn.q_proj.lora_A.default.weight', 'base_model.model.model.layers.1.self_attn.q_proj.lora_B.default.weight', 'base_model.model.model.layers.1.self_attn.v_proj.lora_A.default.weight', 'base_model.model.model.layers.1.self_attn.v_proj.lora_B.default.weight', 'base_model.model.model.layers.2.self_attn.q_proj.lora_A.default.weight', 'base_model.model.model.layers.2.self_attn.q_proj.lora_B.default.weight', 'base_model.model.model.layers.2.self_attn.v_proj.lora_A.default.weight', 'base_model.model.model.layers.2.self_attn.v_proj.l

In [ ]:
# Function to chat with the model
def chat():
    print("🤖 Fine-Tuned TinyLlama Chatbot (type 'exit' to stop)")
    while True:
        user_input = input("You: ")
        if user_input.lower() == "exit":
            print("Goodbye! 👋")
            break

        # Tokenize input
        inputs = tokenizer(user_input, return_tensors="pt").to("cuda")

        # Generate response
        output = model.generate(**inputs, max_length=200, pad_token_id=tokenizer.eos_token_id)
        response = tokenizer.decode(output[0], skip_special_tokens=True)

        print(f"Fine-Tuned TinyLlama: {response}")

# Start chat
chat()

🤖 Fine-Tuned TinyLlama Chatbot (type 'exit' to stop)
You: hello
Fine-Tuned TinyLlama: hello, and then I'll be back.

JASON: (smiling) Hey, I'm glad you're here.

JASON'S MOTHER: (smiling) You're welcome.

INT. JASON'S HOUSE - DAY

Jason is sitting on the couch, watching TV. His mother enters the room.

JASON'S MOTHER: (smiling) Hey, Jason. How was your day?

JASON: (smiling) It was good.

JASON'S MOTHER: (nodding) I'm glad to hear that.

JASON: (looking at his phone) Hey, I'm going to be late for work.

JASON'S MOTHER: (smiling) That's okay. I'll be here.


You: what is the capital of india?
Fine-Tuned TinyLlama: what is the capital of india?
You: hi


In [ ]:
model.save_pretrained("./fine_tuned_tinyllama")
tokenizer.save_pretrained("./fine_tuned_tinyllama")

('./fine_tuned_tinyllama/tokenizer_config.json',
 './fine_tuned_tinyllama/special_tokens_map.json',
 './fine_tuned_tinyllama/tokenizer.model',
 './fine_tuned_tinyllama/added_tokens.json',
 './fine_tuned_tinyllama/tokenizer.json')